In [48]:
from keras.models import *
from keras.layers import *
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tensorflow import keras
from keras import optimizers
import pandas as pd
import numpy as np
import tensorflow as tf


In [15]:
train = pd.read_csv('./data/dacon_g/train_199694.csv', index_col='id')
test = pd.read_csv('./data/dacon_g/test.csv', index_col='id')
sample_submission = pd.read_csv('./data/dacon_g/sample_submission.csv', index_col='id')

In [49]:
train = pd.read_csv('./data/dacon_g/train_10components.csv', index_col='id')
test = pd.read_csv('./data/dacon_g/test_10components.csv', index_col='id')
sample_submission = pd.read_csv('./data/dacon_g/sample_submission.csv', index_col='id')

In [50]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [41]:
train

,type,fiberID,component1,component2,component3,component4,component5,component6,component7,component8,component9,component10,type_num
id,,,,,,,,,,,,,
0,QSO,601,-0.095638,-0.006137,0.010136,-0.001293,0.002862,-0.002194,-0.004993,-0.000719,0.007276,0.005954,8
1,QSO,788,-0.077533,-0.006018,0.005502,-0.002070,0.006184,-0.002220,-0.000992,-0.001525,0.007408,0.007914,8
2,QSO,427,0.043921,-0.003309,-0.006999,0.002722,0.012508,0.000234,0.007171,-0.004103,-0.006349,0.004465,8
3,QSO,864,-0.050537,-0.005554,0.002694,-0.000332,0.004255,-0.001389,0.000907,-0.000651,0.003710,0.005720,8
4,STAR_RED_DWARF,612,-0.075992,-0.004086,0.005564,-0.005199,-0.026765,0.000238,-0.016964,0.015139,0.002675,-0.011342,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,QSO,568,-0.058810,-0.005522,0.002690,-0.000250,0.005573,-0.001221,0.002999,-0.001457,0.003270,0.007117,8
199987,GALAXY,362,-0.072745,-0.001950,0.005270,-0.007974,-0.018186,0.000472,-0.021770,0.001663,-0.006210,-0.020313,6
199988,STAR_RED_DWARF,146,-0.045137,-0.002580,0.002797,-0.003638,-0.027253,0.000309,-0.017495,0.011496,-0.003034,-0.014895,10


In [5]:
train_x = train.drop(columns=['Unnamed: 0','type', 'fiberID','type_num'], axis=1)
train_y = train['type_num']
test_x = test.drop(columns=['Unnamed: 0','fiberID'], axis=1)

In [51]:
train_x = train.drop(columns=['type', 'fiberID','type_num'], axis=1)
train_y = train['type_num']
test_x = test.drop(columns=['fiberID'], axis=1)

In [43]:
scaler = RobustScaler()
# labels = train['type']
# train = train.drop(columns=['fiberID', 'type']) # fiberID는 1000개의 categorical feature이며, 이 커널에서는 무시합니다.

In [44]:
_mat = scaler.fit_transform(train_x)
train_x = pd.DataFrame(_mat, columns=train_x.columns, index=train_x.index)

In [52]:
train_x

,component1,component2,component3,component4,component5,component6,component7,component8,component9,component10
id,,,,,,,,,,
0,-0.095638,-0.006137,0.010136,-0.001293,0.002862,-0.002194,-0.004993,-0.000719,0.007276,0.005954
1,-0.077533,-0.006018,0.005502,-0.002070,0.006184,-0.002220,-0.000992,-0.001525,0.007408,0.007914
2,0.043921,-0.003309,-0.006999,0.002722,0.012508,0.000234,0.007171,-0.004103,-0.006349,0.004465
3,-0.050537,-0.005554,0.002694,-0.000332,0.004255,-0.001389,0.000907,-0.000651,0.003710,0.005720
4,-0.075992,-0.004086,0.005564,-0.005199,-0.026765,0.000238,-0.016964,0.015139,0.002675,-0.011342
...,...,...,...,...,...,...,...,...,...,...
199986,-0.058810,-0.005522,0.002690,-0.000250,0.005573,-0.001221,0.002999,-0.001457,0.003270,0.007117
199987,-0.072745,-0.001950,0.005270,-0.007974,-0.018186,0.000472,-0.021770,0.001663,-0.006210,-0.020313
199988,-0.045137,-0.002580,0.002797,-0.003638,-0.027253,0.000309,-0.017495,0.011496,-0.003034,-0.014895


In [54]:
%%time
model = Sequential()
model.add(Dense(256, kernel_initializer='random_uniform', input_shape=(10,), activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512, kernel_initializer='random_uniform',  activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(19, kernel_initializer='random_uniform', activation='softmax'))

# 3. 모델 학습과정 설정하기
# Adadelta(learning_rate=1.0, rho=0.95)
Adamax = optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
Adam = optimizers.Adam(learning_rate=0.0005,amsgrad=True)
model.compile(optimizer=Adamax, loss='sparse_categorical_crossentropy', metrics=["accuracy"])
 
# 4. 모델 학습시키기
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=40, verbose=1, mode='auto')
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
hist = model.fit(train_x, train_y, epochs=1000, batch_size = 256*2, callbacks=[early_stopping,mc], validation_split=0.2, verbose=2)

Train on 159992 samples, validate on 39999 samples
Epoch 1/1000
 - 19s - loss: 2.1377 - accuracy: 0.3161 - val_loss: 1.7601 - val_accuracy: 0.4241
Epoch 2/1000
 - 19s - loss: 1.5209 - accuracy: 0.4837 - val_loss: 1.3376 - val_accuracy: 0.5520
Epoch 3/1000
 - 19s - loss: 1.3083 - accuracy: 0.5520 - val_loss: 1.1781 - val_accuracy: 0.5992
Epoch 4/1000
 - 19s - loss: 1.1988 - accuracy: 0.5982 - val_loss: 1.0371 - val_accuracy: 0.6640
Epoch 5/1000
 - 19s - loss: 1.0266 - accuracy: 0.6703 - val_loss: 0.8457 - val_accuracy: 0.7303
Epoch 6/1000
 - 19s - loss: 0.8783 - accuracy: 0.7117 - val_loss: 0.7578 - val_accuracy: 0.7506
Epoch 7/1000
 - 19s - loss: 0.7884 - accuracy: 0.7364 - val_loss: 0.6843 - val_accuracy: 0.7692
Epoch 8/1000
 - 19s - loss: 0.7295 - accuracy: 0.7553 - val_loss: 0.6230 - val_accuracy: 0.7922
Epoch 9/1000
 - 19s - loss: 0.6905 - accuracy: 0.7702 - val_loss: 0.5980 - val_accuracy: 0.8025
Epoch 10/1000
 - 19s - loss: 0.6509 - accuracy: 0.7833 - val_loss: 0.5982 - val_accur

 - 19s - loss: 0.4468 - accuracy: 0.8454 - val_loss: 0.4398 - val_accuracy: 0.8487
Epoch 86/1000
 - 19s - loss: 0.4462 - accuracy: 0.8458 - val_loss: 0.4207 - val_accuracy: 0.8541
Epoch 87/1000
 - 19s - loss: 0.4451 - accuracy: 0.8462 - val_loss: 0.4214 - val_accuracy: 0.8542
Epoch 88/1000
 - 19s - loss: 0.4441 - accuracy: 0.8470 - val_loss: 0.4255 - val_accuracy: 0.8518
Epoch 89/1000
 - 19s - loss: 0.4439 - accuracy: 0.8467 - val_loss: 0.4233 - val_accuracy: 0.8545
Epoch 90/1000
 - 19s - loss: 0.4448 - accuracy: 0.8465 - val_loss: 0.4255 - val_accuracy: 0.8517
Epoch 91/1000
 - 19s - loss: 0.4442 - accuracy: 0.8468 - val_loss: 0.4202 - val_accuracy: 0.8536
Epoch 92/1000
 - 19s - loss: 0.4439 - accuracy: 0.8464 - val_loss: 0.4179 - val_accuracy: 0.8542
Epoch 93/1000
 - 19s - loss: 0.4427 - accuracy: 0.8472 - val_loss: 0.4254 - val_accuracy: 0.8531
Epoch 94/1000
 - 19s - loss: 0.4446 - accuracy: 0.8466 - val_loss: 0.4210 - val_accuracy: 0.8549
Epoch 95/1000
 - 19s - loss: 0.4426 - accura

Epoch 169/1000
 - 19s - loss: 0.4233 - accuracy: 0.8526 - val_loss: 0.4061 - val_accuracy: 0.8586
Epoch 170/1000
 - 19s - loss: 0.4207 - accuracy: 0.8530 - val_loss: 0.4051 - val_accuracy: 0.8589
Epoch 171/1000
 - 19s - loss: 0.4218 - accuracy: 0.8528 - val_loss: 0.4058 - val_accuracy: 0.8587
Epoch 172/1000
 - 19s - loss: 0.4194 - accuracy: 0.8526 - val_loss: 0.4103 - val_accuracy: 0.8578
Epoch 173/1000
 - 19s - loss: 0.4214 - accuracy: 0.8532 - val_loss: 0.4038 - val_accuracy: 0.8593
Epoch 174/1000
 - 19s - loss: 0.4194 - accuracy: 0.8528 - val_loss: 0.4116 - val_accuracy: 0.8560
Epoch 175/1000
 - 19s - loss: 0.4196 - accuracy: 0.8531 - val_loss: 0.4076 - val_accuracy: 0.8578
Epoch 176/1000
 - 19s - loss: 0.4197 - accuracy: 0.8533 - val_loss: 0.4024 - val_accuracy: 0.8593
Epoch 177/1000
 - 19s - loss: 0.4207 - accuracy: 0.8533 - val_loss: 0.4023 - val_accuracy: 0.8599
Epoch 178/1000
 - 19s - loss: 0.4213 - accuracy: 0.8529 - val_loss: 0.4142 - val_accuracy: 0.8569
Epoch 179/1000
 - 19

Epoch 253/1000
 - 19s - loss: 0.4096 - accuracy: 0.8561 - val_loss: 0.4000 - val_accuracy: 0.8604
Epoch 254/1000
 - 19s - loss: 0.4094 - accuracy: 0.8562 - val_loss: 0.4044 - val_accuracy: 0.8594
Epoch 255/1000
 - 19s - loss: 0.4090 - accuracy: 0.8564 - val_loss: 0.3994 - val_accuracy: 0.8607
Epoch 256/1000
 - 19s - loss: 0.4088 - accuracy: 0.8564 - val_loss: 0.4036 - val_accuracy: 0.8575
Epoch 257/1000
 - 19s - loss: 0.4079 - accuracy: 0.8570 - val_loss: 0.4035 - val_accuracy: 0.8600
Epoch 258/1000
 - 19s - loss: 0.4087 - accuracy: 0.8566 - val_loss: 0.4015 - val_accuracy: 0.8604
Epoch 259/1000
 - 19s - loss: 0.4060 - accuracy: 0.8567 - val_loss: 0.4047 - val_accuracy: 0.8583
Epoch 260/1000
 - 19s - loss: 0.4087 - accuracy: 0.8563 - val_loss: 0.4009 - val_accuracy: 0.8605
Epoch 261/1000
 - 19s - loss: 0.4076 - accuracy: 0.8565 - val_loss: 0.4007 - val_accuracy: 0.8605
Epoch 262/1000
 - 19s - loss: 0.4072 - accuracy: 0.8570 - val_loss: 0.4058 - val_accuracy: 0.8572
Epoch 263/1000
 - 19

In [ ]:
#  - 17s - loss: 0.3547 - accuracy: 0.8744 - val_loss: 0.3606 - val_accuracy: 0.8755
# Epoch 00109: early stopping
# log-loss : 0.378556
# para :  early - 14, adam - 0.0005/True

# log-loss : ?
# para :  early - 20, adadelta - default
#  - 40s - loss: 0.3701 - accuracy: 0.8714 - val_loss: 0.3662 - val_accuracy: 0.8714
# Epoch 00144: early stopping
# CPU times: user 2h 31min 8s, sys: 2min 49s, total: 2h 33min 57s
# Wall time: 1h 23min 9s

# log-loss : ?
# para :  early - 20, adamax - 0.001
#  - 18s - loss: 0.3536 - accuracy: 0.8749 - val_loss: 0.3590 - val_accuracy: 0.8745
# Epoch 00152: early stopping
# CPU times: user 2h 34min 16s, sys: 2min 42s, total: 2h 36min 59s
# Wall time: 1h 26min 14s



In [30]:

test1 = pd.DataFrame(scaler.transform(test_x), columns=test_x.columns, index=test_x.index)

In [31]:
pred_mat = model.predict(test1)

In [ ]:
pred_mat


In [ ]:
array([[1.9908954e-24, 2.5674998e-18, 3.3080816e-09, ..., 1.0879984e-25,
        4.9456245e-11, 1.1162232e-19],
       [5.8779093e-09, 6.5391893e-11, 0.0000000e+00, ..., 1.7154184e-23,
        1.4704717e-12, 1.1403962e-08],
       [2.0768403e-11, 1.2031060e-16, 8.8013845e-22, ..., 1.9567362e-36,
        1.7418125e-09, 3.6088350e-12],
       ...,
       [1.6718870e-12, 3.9923211e-17, 0.0000000e+00, ..., 8.9288002e-31,
        9.8670749e-10, 8.4885614e-06],
       [5.5168621e-23, 7.0876017e-38, 0.0000000e+00, ..., 0.0000000e+00,
        7.4107750e-27, 3.4055818e-12],
       [2.0796194e-09, 4.9559921e-23, 4.7859279e-26, ..., 0.0000000e+00,
        6.4991754e-27, 3.1902134e-15]], dtype=float32)

In [32]:
submission = pd.DataFrame(data=pred_mat, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('./data/dacon_g/submission_DL_N3.csv', index=True)